In [ ]:
import json
#VERIFICATION
# Specify the path to the COCO formatted output file
output_file_path = r"C:\Users\hp\Documents\EFREI_PFE\15_05_2024\Gait Dataset\New dataset_Gait\DataSet\Canter\Videos\collected Cantercoco_format3.json"  # Change this to your desired output file path

# Load the COCO formatted JSON
with open(output_file_path) as json_file:
    coco_data = json.load(json_file)

# Function to verify the COCO format
def verify_coco_format(coco_data):
    # Check presence of required fields
    required_fields = ["images", "annotations", "categories"]
    for field in required_fields:
        if field not in coco_data:
            print(f"Missing field: {field}")
            return False

    # Check images
    for image in coco_data["images"]:
        if "id" not in image or "file_name" not in image or "height" not in image or "width" not in image:
            print(f"Invalid image entry: {image}")
            return False

    # Check annotations
    for annotation in coco_data["annotations"]:
        if "keypoints" not in annotation or "image_id" not in annotation or "bbox" not in annotation:
            print(f"Invalid annotation entry: {annotation}")
            return False

        # Check bbox format
        if len(annotation["bbox"]) != 4:
            print(f"Invalid bbox for annotation {annotation['id']}: {annotation['bbox']}")
            return False

        # Check keypoints format
        if len(annotation["keypoints"]) % 3 != 0:
            print(f"Invalid keypoints length for annotation {annotation['id']}: {annotation['keypoints']}")
            return False

    # Check categories
    for category in coco_data["categories"]:
        if "id" not in category or "name" not in category or "keypoints" not in category:
            print(f"Invalid category entry: {category}")
            return False

    print("COCO format verification passed.")
    return True

# Run verification
verify_coco_format(coco_data)


COCO format verification passed.


True

In [ ]:
#the official correct for mapping ViTPose JSON output to cvat JSON
import json
import numpy as np

# Specify input and output file paths
input_file_path =r"C:\Users\hp\Downloads\results\results\results_V21.json"
output_file_path = r"C:\Users\hp\Downloads\new_videos\new_videos\results_V21CVAT.json"

# Define the input order (as in your JSON)
input_order = [
    "T1", "T2", "A8", "A7_1", "A7", "A6", "A5", "A3", "A1",
    "A6_1", "A5_1", "A3_1", "A1_1", "A10", "P9", "P9_1",
    "P7", "P5", "P3", "P1", "P7_1", "P5_1", "P3_1", "P1_1",
    "Q1", "Q2", "Q3", "Q4"
]

# Define the desired output order
output_order = [
    "T1", "T2", "A8", "A7", "A6", "A5", "A3", "A1", "A7_1",
    "A6_1", "A5_1", "A3_1", "A1_1", "A10", "P9", "P7",
    "P5", "P3", "P1", "P9_1", "P7_1", "P5_1", "P3_1", "P1_1",
    "Q1", "Q2", "Q3", "Q4"
]

# Create a mapping from input indices to output indices
keypoint_mapping = {input_order.index(kp): output_order.index(kp) for kp in input_order}
print("Keypoint mapping:")
for i, o in keypoint_mapping.items():
    print(f"  Input index {i} ({input_order[i]}) -> Output index {o} ({output_order[o]})")

# Load input JSON from the specified file
with open(input_file_path) as json_file:
    input_json = json.load(json_file)

# Initialize COCO format structure
coco_format = {
    "images": [],
    "annotations": [],
    "categories": [
        {
            "supercategory": "animal",
            "id": 1,
            "name": "horse",
            "keypoints": output_order,
            "skeleton": []
        }
    ]
}

# Populate images and annotations
image_id = 0
annotation_id = 0

for instance_info in input_json["instance_info"]:
    frame_id = instance_info["frame_id"]-1
    file_name = f"frame_{frame_id:04d}.jpg"

    print(f"\nProcessing frame {frame_id}")

    # Add image info
    coco_format["images"].append({
        "file_name": file_name,
        "height": 720,
        "width": 1280,
        "id": image_id
    })

    for instance in instance_info["instances"]:
        print(f"  Processing instance {annotation_id}")
        keypoints = instance["keypoints"]
        mapped_keypoints = [None] * len(output_order)

        print("    Original keypoints:")
        for i, kp in enumerate(keypoints):
            print(f"      {input_order[i]}: {kp}")
            mapped_index = keypoint_mapping[i]
            mapped_keypoints[mapped_index] = kp

        print("    Mapped keypoints:")
        for i, kp in enumerate(mapped_keypoints):
            print(f"      {output_order[i]}: {kp}")

        flattened_keypoints = []
        for kp in mapped_keypoints:
            if kp is None:
                flattened_keypoints.extend([0, 0, 0])  # Add default values for missing keypoints
            else:
                flattened_keypoints.extend([kp[0], kp[1], 2])  # Add X, Y, visibility

        # Get the bounding box
        bbox = instance["bbox"]
        if isinstance(bbox[0], list):
            bbox = bbox[0]  # Flatten if bbox is nested

        # Check bbox length
        if len(bbox) != 4:
            print(f"Warning: Expected bbox with 4 values but got {bbox}. Skipping this instance.")
            continue

        # Add annotation info
        coco_format["annotations"].append({
            "keypoints": flattened_keypoints,
            "image_id": image_id,
            "id": annotation_id,
            "num_keypoints": len(keypoints),
            "iscrowd": 0,
            "category_id": 1,
            "segmentation": [],
            "bbox": bbox
        })

        annotation_id += 1
    image_id += 1

# Save the COCO formatted data to the specified output file
with open(output_file_path, "w") as json_file:
    json.dump(coco_format, json_file, indent=4)

print("\nConversion to COCO format completed successfully.")

Keypoint mapping:
  Input index 0 (T1) -> Output index 0 (T1)
  Input index 1 (T2) -> Output index 1 (T2)
  Input index 2 (A8) -> Output index 2 (A8)
  Input index 3 (A7_1) -> Output index 8 (A7_1)
  Input index 4 (A7) -> Output index 3 (A7)
  Input index 5 (A6) -> Output index 4 (A6)
  Input index 6 (A5) -> Output index 5 (A5)
  Input index 7 (A3) -> Output index 6 (A3)
  Input index 8 (A1) -> Output index 7 (A1)
  Input index 9 (A6_1) -> Output index 9 (A6_1)
  Input index 10 (A5_1) -> Output index 10 (A5_1)
  Input index 11 (A3_1) -> Output index 11 (A3_1)
  Input index 12 (A1_1) -> Output index 12 (A1_1)
  Input index 13 (A10) -> Output index 13 (A10)
  Input index 14 (P9) -> Output index 14 (P9)
  Input index 15 (P9_1) -> Output index 19 (P9_1)
  Input index 16 (P7) -> Output index 15 (P7)
  Input index 17 (P5) -> Output index 16 (P5)
  Input index 18 (P3) -> Output index 17 (P3)
  Input index 19 (P1) -> Output index 18 (P1)
  Input index 20 (P7_1) -> Output index 20 (P7_1)
  Inpu

In [ ]:
import json
import numpy as np
#this script is for the case where there is inversive legs all along the annotation
# Specify input and output file paths
input_file_path = r"C:\Users\hp\Downloads\results\results\results_V29.json"
output_file_path = r"C:\Users\hp\Downloads\results\results\results_V29CVAT1.json"

# Define the input order (as in your JSON)
input_order = [
    "T1", "T2", "A8", "A7_1", "A7", "A6", "A5", "A3", "A1",
    "A6_1", "A5_1", "A3_1", "A1_1", "A10", "P9", "P9_1",
    "P7", "P5", "P3", "P1", "P7_1", "P5_1", "P3_1", "P1_1",
    "Q1", "Q2", "Q3", "Q4"
]

# Define the desired output order (unchanged)
output_order = [
    "T1", "T2", "A8", "A7", "A6", "A5", "A3", "A1", "A7_1",
    "A6_1", "A5_1", "A3_1", "A1_1", "A10", "P9", "P7",
    "P5", "P3", "P1", "P9_1", "P7_1", "P5_1", "P3_1", "P1_1",
    "Q1", "Q2", "Q3", "Q4"
]

# Pairs of keypoints to swap
#you can inspect the annotation and based on that choose which keypoints to be swapped
swap_pairs = [
    ('A7', 'A7_1'),
    ('A5', 'A5_1'),
    ('A3', 'A3_1'),
    ('A1', 'A1_1'),
    ('A6', 'A6_1'),
    #('P9', 'P9_1'),
    #('P7', 'P7_1'),
    #('P5', 'P5_1'),
    #('P3', 'P3_1'),
    #('P1', 'P1_1')
]

# Create a mapping from input indices to output indices
keypoint_mapping = {input_order.index(kp): output_order.index(kp) for kp in input_order}

print("Keypoint mapping:")
for i, o in keypoint_mapping.items():
    print(f"  Input index {i} ({input_order[i]}) -> Output index {o} ({output_order[o]})")

# Load input JSON from the specified file
with open(input_file_path) as json_file:
    input_json = json.load(json_file)

# Initialize COCO format structure
coco_format = {
    "images": [],
    "annotations": [],
    "categories": [
        {
            "supercategory": "animal",
            "id": 1,
            "name": "horse",
            "keypoints": output_order,
            "skeleton": []
        }
    ]
}

# Populate images and annotations
image_id = 0
annotation_id = 0

def swap_keypoints(keypoints, swap_pairs):
    """Swap the values for the given pairs of keypoints."""
    print("    Original keypoints before swap:")
    for i, kp in enumerate(keypoints):
        print(f"      {input_order[i]}: {kp}")

    for kp1, kp2 in swap_pairs:
        idx1 = input_order.index(kp1)
        idx2 = input_order.index(kp2)
        keypoints[idx1], keypoints[idx2] = keypoints[idx2], keypoints[idx1]

    print("    Keypoints after swap:")
    for i, kp in enumerate(keypoints):
        print(f"      {input_order[i]}: {kp}")

    return keypoints

for instance_info in input_json["instance_info"]:
    frame_id = instance_info["frame_id"]-1
    file_name = f"frame_{frame_id:04d}.jpg"

    print(f"\nProcessing frame {frame_id}")

    # Add image info
    coco_format["images"].append({
        "file_name": file_name,
        "height": 720,
        "width": 1280,
        "id": image_id
    })

    for instance in instance_info["instances"]:
        print(f"  Processing instance {annotation_id}")
        keypoints = instance["keypoints"]

        # Swap keypoints for specific pairs
        swapped_keypoints = swap_keypoints(keypoints, swap_pairs)

        # Map the swapped keypoints to the output order
        mapped_keypoints = [None] * len(output_order)
        for i, kp in enumerate(swapped_keypoints):
            mapped_index = keypoint_mapping[i]
            mapped_keypoints[mapped_index] = kp

        # Flatten the keypoints for COCO format
        flattened_keypoints = []
        for kp in mapped_keypoints:
            if kp is None:
                flattened_keypoints.extend([0, 0, 0])  # Add default values for missing keypoints
            else:
                flattened_keypoints.extend([kp[0], kp[1], 2])  # Add X, Y, visibility

        # Get the bounding box
        bbox = instance["bbox"]
        if isinstance(bbox[0], list):
            bbox = bbox[0]  # Flatten if bbox is nested

        # Check bbox length
        if len(bbox) != 4:
            print(f"Warning: Expected bbox with 4 values but got {bbox}. Skipping this instance.")
            continue

        # Add annotation info
        coco_format["annotations"].append({
            "keypoints": flattened_keypoints,
            "image_id": image_id,
            "id": annotation_id,
            "num_keypoints": len(keypoints),
            "iscrowd": 0,
            "category_id": 1,
            "segmentation": [],
            "bbox": bbox
        })

        annotation_id += 1
    image_id += 1

# Save the COCO formatted data to the specified output file
with open(output_file_path, "w") as json_file:
    json.dump(coco_format, json_file, indent=4)

print("\nConversion to COCO format completed successfully.")


Keypoint mapping:
  Input index 0 (T1) -> Output index 0 (T1)
  Input index 1 (T2) -> Output index 1 (T2)
  Input index 2 (A8) -> Output index 2 (A8)
  Input index 3 (A7_1) -> Output index 8 (A7_1)
  Input index 4 (A7) -> Output index 3 (A7)
  Input index 5 (A6) -> Output index 4 (A6)
  Input index 6 (A5) -> Output index 5 (A5)
  Input index 7 (A3) -> Output index 6 (A3)
  Input index 8 (A1) -> Output index 7 (A1)
  Input index 9 (A6_1) -> Output index 9 (A6_1)
  Input index 10 (A5_1) -> Output index 10 (A5_1)
  Input index 11 (A3_1) -> Output index 11 (A3_1)
  Input index 12 (A1_1) -> Output index 12 (A1_1)
  Input index 13 (A10) -> Output index 13 (A10)
  Input index 14 (P9) -> Output index 14 (P9)
  Input index 15 (P9_1) -> Output index 19 (P9_1)
  Input index 16 (P7) -> Output index 15 (P7)
  Input index 17 (P5) -> Output index 16 (P5)
  Input index 18 (P3) -> Output index 17 (P3)
  Input index 19 (P1) -> Output index 18 (P1)
  Input index 20 (P7_1) -> Output index 20 (P7_1)
  Inpu